In [1]:
#resnet152
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report, confusion_matrix

import torch


# Config
num_classes=5
data_dir = "./datasetaptos19"  # Your dataset path
batch_size = 102
num_workers = 2
num_epochs = 3
delta = 1e-4
max_grad_norm = 1.0
learning_rate = 1e-3

#, 1.0, 2.0, 4.0, 8.0, 16.0


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/val", transform=transform)
class_names = val_dataset.classes
# DataLoaders
def get_dataloaders():
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader

# Track ε vs accuracy
results = []

In [3]:
epsilon_target= 2.0
print(f"\n=== Training with ε target: {epsilon_target} ===")

    # Reload model for each run
model = models.resnet18(pretrained=True)
    #model = models.resnet152(pretrained=True)
    #for param in model.parameters():
        #param.requires_grad = False
for name, param in model.named_parameters():
     if not name.startswith("fc"):  # 'fc' is the classifier head in ResNet
        param.requires_grad = False

# Optional: Replace the classifier head
num_classes = 5  # or whatever your task requires
    #model.fc = nn.Linear(model.fc.in_features, num_classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
    




criterion = nn.CrossEntropyLoss()
model = model.to(device)
    #print(model)
    # Optimizer and loss
    #optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    

    # DataLoader
train_loader, val_loader = get_dataloaders()

    # DP setup
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        target_epsilon=epsilon_target,
        target_delta=delta,
        epochs=num_epochs,
        max_grad_norm=max_grad_norm,
    )

    # Train loop
for epoch in range(num_epochs):
        model.train()
        print("epoch:",epoch)
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)    
            loss.backward()
            optimizer.step()

    # Evaluate
model.eval()
all_preds = []
all_labels = []
test_loss=0
with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss =test_loss+ criterion(outputs, labels).item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

   
test_loss /= len(val_loader)
acc = accuracy_score(all_labels, all_preds)
    #eps_spent, best_alpha = privacy_engine.get_privacy_spent(delta)
epsilon = privacy_engine.accountant.get_epsilon(delta)
print(f"Final ε: {epsilon:.2f}, Accuracy: {acc*100:.2f}%")
    
    
print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {acc:.4f}\n')
print("Detailed Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names, zero_division=0))
    
    

results.append((epsilon, acc))
print("Epsilon=",epsilon,"Accuracy=", acc)


=== Training with ε target: 2.0 ===


C:\Users\User\anaconda3\envs\torch25\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\anaconda3\envs\torch25\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\User\anaconda3\envs\torch25\lib\site-packages\opacus\privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
C:\Users\User

epoch: 0
epoch: 1
epoch: 2
Final ε: 2.00, Accuracy: 55.19%

Test set: Average loss: 1.2551, Accuracy: 0.5519

Detailed Classification Report:
              precision    recall  f1-score   support

          C0       0.55      1.00      0.71       199
          C1       0.00      0.00      0.00        30
          C2       0.60      0.03      0.07        87
          C3       0.00      0.00      0.00        17
          C4       0.00      0.00      0.00        33

    accuracy                           0.55       366
   macro avg       0.23      0.21      0.16       366
weighted avg       0.44      0.55      0.40       366

Epsilon= 1.9950622248238812 Accuracy= 0.5519125683060109
